In [1]:
# Import necessary libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime

def get_match_data(start_date,time_now_formatted,devengine):


    # Get historical match data on hard surface between start date and yesterday
    elo_hard = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    # Get historical match data on clay surface between start date and yesterday
    elo_clay = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    #Get today's matches on hard surface that haven't yet been resulted
    elo_data_hard =pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date like '{time_now_formatted}' and resulted like 'False'",
        con=devengine,)

    #Get today's matches on clay surface that haven't yet been resulted
    elo_data_clay =pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date like '{time_now_formatted}' and resulted like 'False'",
        con=devengine,)

    return elo_hard, elo_clay, elo_data_hard, elo_data_clay

# Connect to SQLite database using SQLAlchemy's create_engine
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
# Get current date and time
time_now = datetime.datetime.now()

# Format current date as string in YYYY-MM-DD format
time_now_formatted = time_now.strftime("%Y-%m-%d")

# Get the start date two years ago from today
today =time_now
two_years_ago = (today - datetime.timedelta(days=365*2)).strftime("%Y-%m-%d")

elo_hard, elo_clay, elo_data_hard, elo_data_clay = get_match_data(two_years_ago,time_now_formatted,devengine)

def get_player_record(player, opponent_rank, history, range_low,range_high,auto):
    if auto:
        opponent_rank_low = opponent_rank - range_low
        opponent_rank_high = opponent_rank + range_high
    else:
        opponent_rank_low = range_low
        opponent_rank_high = range_high

    player_history = history[
        (
            (history["Fav"] == player)
            & (
                (history["Dog_Rank"] > opponent_rank_low)
                & (history["Dog_Rank"] < opponent_rank_high)
            )
        )
        | (
            (history["Dog"] == player)
            & (
                (history["Fav_Rank"] > opponent_rank_low)
                & (history["Fav_Rank"] < opponent_rank_high)
            )
        )
    ]
    if player_history.empty == False:
        result = float(
            len(player_history[player_history["Winner"] == player])
            / len(player_history)
        )
        return result, len(player_history)
    else:
        return 0, 0

def get_filtered_data(elo_data, elo):
    result_df = pd.DataFrame()
    for _, row in elo_data.sort_values(by="Time").iterrows():
        low_limit = 50
        high_limit = 50

        fav_percent, games = get_player_record(row.Fav, row.Dog_Rank, elo_hard, low_limit, high_limit, True)
        count = 0
        while games < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            fav_percent, games = get_player_record(row.Fav, row.Dog_Rank, elo_hard, low_limit, high_limit, True)

        low_limit = 50
        high_limit = 50
        dog_percent, games2 = get_player_record(row.Dog, row.Fav_Rank, elo_hard, low_limit, high_limit, True)
        count = 0
        while games2 < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            dog_percent, games2 = get_player_record(row.Dog, row.Fav_Rank, elo_hard, low_limit, high_limit, True)

        if games > 4 and games2 > 4:
            temp_df = pd.DataFrame(
                {
                    "Time": [row.Time],
                    "Fav_Odds": [row.Fav_Odds],
                    "Dog_Odds": [row.Dog_Odds],
                    "Fav": [row.Fav],
                    "Elo_Fav": [row.Elo_Fav],
                    "Fav_Record": ["{:.0%}".format(fav_percent)],
                    "Fav_Games": [games],
                    "Dog": [row.Dog],
                    "Dog_Odds": [row.Dog_Odds],
                    "Dog_Record": ["{:.0%}".format(dog_percent)],
                    "Dog_Games": [games2],
                    "fav_percent": [fav_percent],
                    "dog_percent": [dog_percent],
                    "Sex": [row.Sex],
                }
            )
            result_df = pd.concat([result_df, temp_df])
    return result_df

results_hard=get_filtered_data(elo_data_hard,elo_hard)
results_clay=get_filtered_data(elo_data_clay,elo_clay)
def process_serving_data(result_df):
    try:
        # Try to read the 'servers_today.xlsx' file
        serving = pd.read_excel('servers_today.xlsx')

        # Try to read the 'servers_today_womens.xlsx' file
        serving_womens = pd.read_excel('servers_today_womens.xlsx')

        # If 'serving_womens' dataframe is not empty, concatenate with 'serving' dataframe
        if serving_womens.empty == False:
            serving = pd.concat([serving, serving_womens])
        else:
            serving = serving

        # Drop the 'Time' column from the 'serving' dataframe
        serving = serving.drop(columns='Time')
    except FileNotFoundError as e:
        # If either of the excel files is not found, print an error message and set serving to None
        print("The required excel file could not be found.")
        print("Error:", e)
        serving = None

    if serving is not None:
        try:
            # Try to merge the 'result_df' and 'serving' dataframes on the 'Fav' and 'Dog' columns
            result = pd.merge(result_df, serving, how='left', left_on=['Fav','Dog'], right_on=['Fav','Dog'])

            # Set 'final_hard' to the 'result' dataframe
            result_serving = result
        except Exception as e:
            # If an error occurs during merging, print an error message and set both 'result' and 'final_hard' to None
            print("Error occured while merging the dataframes.")
            print("Error:", e)
            result = None
            result_serving = None
    else:
        result = None
        result_serving = None
        
    return result, result_serving


_,serving_hard=process_serving_data(results_hard)
_,serving_clay=process_serving_data(results_clay)

elo_hardxx = pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '2022-01-01' and Date not like '{time_now_formatted}' --and Fav_odds>1.2 and Fav_odds<1.4 ",
    con=devengine,)

data_concat = pd.DataFrame(columns=['Date', 'Player', 'Odds', 'Win/Loss'])
for dataset_type in [('Winner', 'Win'), ('Loser', 'Loss')]:
    df = elo_hard[['Date', dataset_type[0], f"{dataset_type[0]}_Odds"]].copy()
    df['Player'] = df[dataset_type[0]]
    df['Odds'] = df[f"{dataset_type[0]}_Odds"]
    df['Win/Loss'] = dataset_type[1]
    df.drop(columns=[dataset_type[0], f"{dataset_type[0]}_Odds"], inplace=True)
    data_concat = pd.concat([data_concat, df])
data_concat = data_concat.sort_index()
data_concat['Odds'] = data_concat.Odds.astype(float)

def analyse_matchups(result_df,data_concat):
    for _,matchup in result_df.iterrows():
        player1=matchup.Fav
        player2=matchup.Dog
        player1_odds=float(matchup.Fav_Odds)
        player1_odds_hi=player1_odds+0.2
        player1_odds_lo=1
        player2_odds=float(matchup.Dog_Odds)
        player2_odds_hi=player2_odds+0.5
        player2_odds_lo=2
        player1=data_concat[data_concat['Player']==player1].copy()
        player2=data_concat[data_concat['Player']==player2].copy()
        player2=player2[(player2['Odds']>player2_odds_lo)&(player2['Odds']<player2_odds_hi)]
        if len(player2)>0:
            winperc2=len(player2[player2['Win/Loss']=='Win'])/len(player2)
        else:
            winperc2=0
        player1=player1[(player1['Odds']>player1_odds_lo)&(player1['Odds']<player1_odds_hi)]
        if len(player1)>0:
            winperc1=len(player1[player1['Win/Loss']=='Win'])/len(player1)
        else:
            winperc1=0
        if len(player1)>5 and len(player2)>3:
            print(matchup.Time,f"{matchup.Fav} ({round(player1_odds_lo,2)}-->{round(player1_odds_hi,2)})",f"{matchup.Dog} ({round(player2_odds_lo,2)}-->{round(player2_odds_hi,2)})")
            print(len(player1),winperc1,len(player2),winperc2)

analyse_matchups(serving_hard,data_concat)
final_hard=serving_hard

Error occured while merging the dataframes.
Error: 'Fav'
20:00 Antoine Escoffier (1-->1.92) Zdenek Kolar (2-->2.54)
45 0.8 12 0.5
21:00 Belinda Bencic (1-->1.53) Shelby Rogers (2-->3.85)
34 0.8823529411764706 29 0.5517241379310345
21:30 Joris De Loore (1-->1.88) Cem Ilkel (2-->2.62)
28 0.8214285714285714 14 0.42857142857142855
22:00 Francesco Maestrelli (1-->1.74) Riccardo Bonadio (2-->2.89)
13 0.6153846153846154 8 0.625
22:30 Qinwen Zheng (1-->1.92) Daria Kasatkina (2-->2.61)
33 0.7575757575757576 14 0.5
23:00 Liam Broady (1-->1.48) Evan Furness (2-->4.03)
28 0.8214285714285714 14 0.21428571428571427
23:00 Marketa Vondrousova (1-->1.33) Dalma Galfi (2-->6.27)
17 0.8235294117647058 15 0.3333333333333333
23:30 Quentin Halys (1-->1.98) Arthur Fils (2-->2.54)
71 0.7464788732394366 10 0.3
23:30 Stefano Travaglia (1-->1.8) Alessandro Giannessi (2-->2.76)
21 0.7142857142857143 4 0.25


In [2]:
final_hard['Fav_Odds']=final_hard['Fav_Odds'].astype(float)

In [3]:
final_hard[(final_hard['dog_percent']>0.6)&(final_hard['fav_percent']<0.4)&(final_hard['Fav_Odds']<1.8)]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%


In [4]:
#Dog
final_hard[(final_hard['dog_percent']>0)&(final_hard['dog_percent']<0.6)&(final_hard['fav_percent']>0.2)&(final_hard['fav_percent']<0.3) & (final_hard['Sex']=='Mens')]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%


In [5]:
#Fav Pickem
final_hard[(final_hard['Fav_Odds']>1.8)& (final_hard['Sex']=='Mens')]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%


In [6]:
#Fav Low Odds
final_hard[(final_hard['dog_percent']>0)&(final_hard['dog_percent']<0.5)&(final_hard['fav_percent']>0.8)&(final_hard['Fav_Odds']>1.19)&(final_hard['Fav_Odds']<1.3)& (final_hard['Sex']=='Mens')]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%
5,23:00,1.28,3.53,Liam Broady,Liam Broady,85%,26,Evan Furness,43%,14,0.846154,0.428571,Mens,76.0,29.0,71.0,18.0


In [7]:
#Fav Low Odds
final_hard[(final_hard['dog_percent']<0.9)&(final_hard['dog_percent']>0.5)&(final_hard['fav_percent']>0.8)&(final_hard['Fav_Odds']>1.19)&(final_hard['Fav_Odds']<1.3)& (final_hard['Sex']=='Mens')]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%


In [8]:
for _,i in elo_data_hard.iterrows():
    check1=elo_hard[((elo_hard['Winner']==i.Winner)&(elo_hard['Loser']==i.Loser))|((elo_hard['Loser']==i.Winner)&(elo_hard['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Quentin Halys beat Arthur Fils
Belinda Bencic beat Shelby Rogers


In [9]:
for _,i in elo_data_clay.iterrows():
    check1=elo_clay[((elo_clay['Winner']==i.Winner)&(elo_clay['Loser']==i.Loser))|((elo_clay['Loser']==i.Winner)&(elo_clay['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

In [10]:
#Women Low Odds Fav
final_hard[((final_hard['dog_percent']>0.2)&(final_hard['fav_percent']<0.7)&(final_hard['Fav_Odds']>1.19)&(final_hard['Fav_Odds']<1.3)&(final_hard['Sex']=='Womens'))|((final_hard['dog_percent']<0.4)&(final_hard['fav_percent']>0.6)&(final_hard['Fav_Odds']>1.19)&(final_hard['Fav_Odds']<1.3)&(final_hard['Sex']=='Womens'))]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%


In [11]:
#Women Dog
final_hard[((final_hard['dog_percent']>0.5)&(final_hard['fav_percent']<0.7)&(final_hard['Sex']=='Womens'))|((final_hard['dog_percent']>0.6)&(final_hard['fav_percent']<0.6)&(final_hard['Sex']=='Womens'))]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%
6,23:00,1.13,5.77,Marketa Vondrousova,Marketa Vondrousova,65%,17,Dalma Galfi,60%,15,0.647059,0.6,Womens,72.9,31.8,67.1,34.0


In [12]:
print(final_hard[['Time','Dog']][(final_hard['dog_percent']>0.2)&(final_hard['dog_percent']<0.3)&(final_hard['fav_percent']<0.3)&(final_hard['fav_percent']>0.2)])
print(final_hard[['Time','Dog']][(final_hard['dog_percent']>0.5)&(final_hard['dog_percent']<0.6)&(final_hard['fav_percent']<0.8)&(final_hard['fav_percent']>0.7)])
print(final_hard[['Time','Dog']][(final_hard['dog_percent']>0.6)&(final_hard['dog_percent']<0.7)&(final_hard['fav_percent']<0.2)&(final_hard['fav_percent']>0.1)])
print(final_hard[['Time','Dog']][(final_hard['dog_percent']>0.7)&(final_hard['dog_percent']<0.8)&(final_hard['fav_percent']<0.6)&(final_hard['fav_percent']>0.5)])

Empty DataFrame
Columns: [Time, Dog]
Index: []
Empty DataFrame
Columns: [Time, Dog]
Index: []
Empty DataFrame
Columns: [Time, Dog]
Index: []
Empty DataFrame
Columns: [Time, Dog]
Index: []


In [13]:
final_hard

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%
0,20:00,1.72,2.04,Antoine Escoffier,Antoine Escoffier,53%,17,Zdenek Kolar,42%,12,0.529412,0.416667,Mens,0.0,14.0,78.0,0.0
1,21:00,1.33,3.35,Belinda Bencic,Belinda Bencic,61%,41,Shelby Rogers,50%,34,0.609756,0.500000,Womens,77.1,27.0,71.8,31.6
2,21:30,1.68,2.12,Joris De Loore,Joris De Loore,91%,11,Cem Ilkel,47%,17,0.909091,0.470588,Mens,0.0,11.0,63.0,0.0
3,22:00,1.54,2.39,Francesco Maestrelli,Francesco Maestrelli,67%,12,Riccardo Bonadio,50%,10,0.666667,0.500000,Mens,67.0,0.0,0.0,8.0
4,22:30,1.72,2.11,Qinwen Zheng,Qinwen Zheng,55%,20,Daria Kasatkina,48%,44,0.550000,0.477273,Womens,75.2,47.1,60.9,26.4
5,23:00,1.28,3.53,Liam Broady,Liam Broady,85%,26,Evan Furness,43%,14,0.846154,0.428571,Mens,76.0,29.0,71.0,18.0
6,23:00,1.13,5.77,Marketa Vondrousova,Marketa Vondrousova,65%,17,Dalma Galfi,60%,15,0.647059,0.600000,Womens,72.9,31.8,67.1,34.0
7,23:30,1.78,2.04,Quentin Halys,Arthur Fils,64%,22,Arthur Fils,50%,10,0.636364,0.500000,Mens,84.0,13.0,75.0,15.0
8,23:30,1.60,2.26,Stefano Travaglia,Stefano Travaglia,38%,13,Alessandro Giannessi,38%,8,0.384615,0.375000,Mens,75.0,18.0,75.0,13.0
9,23:30,1.55,2.37,Ryan Peniston,Giovanni Fonio,91%,11,Giovanni Fonio,50%,10,0.909091,0.500000,Mens,80.0,0.0,0.0,19.0


In [14]:
results_clay

""


In [15]:
print(final_hard['Dog'][(final_hard['dog_percent']>0.1)&(final_hard['dog_percent']<0.2)&(final_hard['fav_percent']<0.6)&(final_hard['fav_percent']>0.5)])
print(final_hard['Dog'][(final_hard['dog_percent']>0.5)&(final_hard['dog_percent']<0.6)&(final_hard['fav_percent']<0.8)&(final_hard['fav_percent']>0.7)])
print(final_hard['Dog'][(final_hard['dog_percent']>0.6)&(final_hard['dog_percent']<0.7)&(final_hard['fav_percent']<0.4)&(final_hard['fav_percent']>0.3)])
print(final_hard['Dog'][(final_hard['dog_percent']>0.6)&(final_hard['dog_percent']<0.7)&(final_hard['fav_percent']<0.7)&(final_hard['fav_percent']>0.6)])
print(final_hard['Dog'][(final_hard['dog_percent']>0.6)&(final_hard['dog_percent']<0.7)&(final_hard['fav_percent']<0.8)&(final_hard['fav_percent']>0.7)])
print(final_hard['Dog'][(final_hard['dog_percent']>0.7)&(final_hard['dog_percent']<0.8)&(final_hard['fav_percent']<0.4)&(final_hard['fav_percent']>0.3)])

Series([], Name: Dog, dtype: object)
Series([], Name: Dog, dtype: object)
Series([], Name: Dog, dtype: object)
Series([], Name: Dog, dtype: object)
Series([], Name: Dog, dtype: object)
Series([], Name: Dog, dtype: object)


In [16]:
final_hard[(final_hard['Fav_Serve%']<final_hard['Dog_Serve%'])&(final_hard['Dog_Return%']>20)&(final_hard['Fav_Serve%']>0)]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%
